## Lendingclub.com Loan Analysis


Lendingclub.com is a peer to peer lending platform that connects investors with borrowers.  A person needing money applies for a personal loan through Lendingclub.com's website.  If approved, the loan is listed on Lendingclub's website where investors can invest in the loans in $25 increments.
### Problem:
#### Investing in consumer debt is inherently risky!
In order to minimize the risk for the investor.  I will use the loan data available from Lendingclub.com and perform analysis on the dataset to predict whether a loan will will default.


#### Data:
All Loan data can be downloaded directly from lendingclub.com website [HERE](https://www.lendingclub.com/info/download-data.action)


#### Data Cleaning:
#### Jupyter notebook file [HERE](https://github.com/petehwu/capstone/blob/master/ipynb_final/Capstone_Data_Cleaning_09092017.ipynb)
I downloaded a total of 10 files from Lendingclub.com's website.  This represents all approved loan data from 2007 to Q2 of 2017.  Initial dataset had 143 columns(features) and 1,524,098 rows of raw data.
I took the following steps to clean the data
- Combined the 10 files into 1 DataFrame and created a new .csv file which contains all the raw data in 1 file that is easier to use.
- Deleted 21 rows of data that did not have a loan amount(summary data from the .csv files)
- For my analysis, I'm only interesed in Loans that are either Fully Paid or Charged off so I deleted any records that were not in either of those 2 categories.  NOTE: Some rows were in "Default" status.  I decided to re-categorize those as "Charged Off" since those loans will most likely become "Charged Off" in the future.
- Over 99% of the loans were of the type "Individual" so I deleted any rows that were not "Individual" type.  
- I identified 54 columns that either had no data or are data that were captured after the loan was funded.  These columns will not be useful for my model so they were deleted.
- some numeric and date fields had text in them(i.e loan term had 36 months instead of just 36 and interest rate had 10% instead of .10) so I removed the text and changed the columns from object to numeric or datetime type.  
- Did a check on the remaining columns and deleted any columns where more than 50% of the data were missing.
- For the columns that remain, fill in any missing values with either 0 or mean value depending on which makes more sense for that particular column
- Created a new column that is the mean of fico_range_low and fico_range_high and then deleted fico_range_low and fico_range_high
- Created a new column that is the number of days from the issue_date of the loan to the date when applicant first obtained any kind of credit.


##### Exploratory Data Analysis:
#### Jupyter notebook file [HERE](https://github.com/petehwu/capstone/blob/master/ipynb_final/Capstone_EDA_09092017.ipynb)
After data cleaning, I ended up with 68 columns and 665556 rows of data. 
- I plotted the number of Charged Off versus Fully Paid Loans.

<img src="https://raw.githubusercontent.com/petehwu/capstone/master/image_files/default_comparison.png" width="400" height="500" />

- I wanted to look at actual Return On Investment Percentage for different Loan Terms.
    - 36 month Loans
<img src="https://raw.githubusercontent.com/petehwu/capstone/master/image_files/36_month_roi.png" width="400" height="400" />

    - 60 month Loans
<img src="https://raw.githubusercontent.com/petehwu/capstone/master/image_files/60_month_roi.png" width="400" height="500" />

-  I also wanted to calculate the theoretical Return On Investment possible:
 - For 36 months loans the mean interest rate charged is about 12.45% which translates to a 20% theoretical Return on Investment if all investments are fully paid due to compounding interest.

 - For 60 months loans the mean interest rate charged is about 17.02% which translates to a 49% theoretical Return on Investment if all investments are fully paid due to compounding interest.

#### I decide to focus my analysis on 60 months loans because of the higher potential Return On Investment and higher risk involved.


### All analysis from this point on will be for 60 months loans only

- Interest Rate charged by Loan Grade
<img src="https://raw.githubusercontent.com/petehwu/capstone/master/image_files/60_month_interest_rate_by_grade.png" width="300" height="400" />

- Default Rate by Loan Grade
<img src="https://raw.githubusercontent.com/petehwu/capstone/master/image_files/60_month_default_rate_by_grade.png" width="250" height="350" />

#### After looking at the Interest Rate and Default Rate plots from above.  I decide to further narrow down my analysis to Loan grades C-G only since those loans have the highest potential return and highest risk.  For Loan Grades C-G the default rate is approximately 35.7%


Plot showing 60 months loan default rate by state.  Darker color means higher default rate.  From the plot, Mississippi and Oklahoma has the highest default rates and Colorado has the lowest default rate.

In [1]:
from IPython.display import IFrame
IFrame('https://plot.ly/~petehwu/1.embed', width=700, height=450)


I decided to drop some more columns:
- dropped the 'term' column because I am only considering 60 months loans(valus is constant in column)
- dropped the 'grade' column because the information is captured in the "sub_grade" column
- dropped the 'installment' column because the payment installment is directly related to loan amount and interest rate
- dropped the 'emp_title' and 'title' columns because those columns seem to be free text input and due to lack of time I am not prepared to perform analysis on the text input but this might be interesting analysis for future iteration.


After plotting some distribution plots and pair plots, I saw some outliers in the data and decided for the continuous columns, delete any rows where the data is more than 3 standard deviations away from the mean.  

#### Modeling
#### Jupyter notebook file [HERE](https://github.com/petehwu/capstone/blob/master/ipynb_final/Capstone__Modeling_09102017.ipynb)


- I decided to use ADABoosting classifier as my model instead of Logistic Regression because my feature space is quite large due to a large number of categorical features and my reserach indicates Logistic Regression do not work as well when the feature space is large and Decision Tree models work better when there is a large number of categorical features. 
- I did a 70/30 stratified split to rserve 30% of data for testing
- I decided to use ADABoosting Classifier to model the data.
- I wanted to run the model using the dataset WITH outliers and WITHOUT outliers to compare results
- Initial result were Precision of .6864 and specificity: 0.27 which is just slightly better than Lendingclubs
 historical results.  I then ran some other combinations of datasets as summarized below

| DataSet Used                             |Precision|Recall|Specificity|Accuracy|
|------------------------------------------|----|----|----|----|
| With Outlier data and no oversampling    |0.6864|0.8854|0.27|0.666|
| With Outlier data and oversampling       |0.6911| 0.8588|0.3071|0.6621|
| Without Outlier data and no oversampling |0.6805|0.8829|0.2676|0.6605|
| Without Outlier data and oversampling    |0.6838|0.8631|0.2946|0.6576|

- Looking at the results, the models using the oversampled datatset did better than the model not using the oversampled dataset.  This makes sense because my original dataset is somewhat unbalanced(many more True than False)  and you can see the Specificity scores are fairly low.  using the oversampled dataset which is more balanced the Specificity increased a bit.  What was somewhat surprising to me was that the dataset containing the outliers performed very slightly better than the dataset not containing outliers.  I feel this is because removing the outliers decreased my observations by too much and thus causing the model to not perform as well.  I expect that as more data becomes available, the model with outliers removed and oversampling to balance the dataset will start to perform better.  

#### Conclusion:
- Historical Mean Default rate for 60 month loans(grade C-G) is approximately 36%  which can be thought of as Lendingclub's predictions having a precision of .64( 64/(64+36)).  My best model has a precision of .69 which is just slightly better than LendingClubs prediction.
-  This tells me that LendingClubs pre-screening process is doing a pretty good job of predicting which borrowers might default given the available data and the results cannot be drastically improved easily with existing available data.
- Using feature_importance parameter, the following are the features that are most important
<img src="https://raw.githubusercontent.com/petehwu/capstone/master/image_files/feature_importance.png" width="300" height="400" />